In [1]:
import dimelo as dm
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import pysam
import time
import importlib
import pyBigWig
import numpy as np

In [30]:
##This dataset uses v1.0 genome
#~31 min on savio3 for chr1 with 24 cores, 29min with 32
megalodon_bam_filepath = '/clusterfs/nilah/oberon/dimelo_dev_source/test_inputs/deep_ctcf_mod_mappings_merge.sorted.bam'

#This dataset uses v1.1 genome
#~21 min on savio3 for chr1 with 32 cores
two_color_r9 = '/clusterfs/nilah/oberon/datasets/dimelo_mA-mGpC/lmnb1-accessibility_20220214_megalodon/barcode10_rabbit-abcam/mod_mappings.10.sorted.bam'

#This dataset uses v1.1 genome
#~21 min on savio3 for chr1 with 32 cores
two_color_redux = '/clusterfs/nilah/oberon/datasets/dimelo_mA-mGpC/20230702_jm_lmnb1_acessibility_redux/megalodon_all_context/mod_mappings.01.sorted.bam'

#This dataset uses v1.0 genome
#22 min to savio3
hia_bam_filepath = '/clusterfs/nilah/ayesha/dimelo/prod_free_Hia5_mod_mappings.sorted.bam'

#This dataset uses v1.1 genome
#~36 min on savio3 for chr1 with 24 cores
histone_1_filepath = '/global/scratch/projects/vector_streetslab/amaslan/dimelo_bams/protocol_paper/promethion/streets1_all.mod_mappings.sorted.bam'
histone_2_filepath = '/global/scratch/projects/vector_streetslab/amaslan/dimelo_bams/protocol_paper/promethion/streets2_all.mod_mappings.sorted.bam'
histone_3_filepath = '/global/scratch/projects/vector_streetslab/amaslan/dimelo_bams/protocol_paper/promethion/streets3_all.mod_mappings.sorted.bam'

output_dir = '/global/scratch/users/dixonluinenburg/Parallelization_test_outputs/'
sql_output = 'test_output'
hp1_sample_name = 'gm12878_ctcf_hp1'
genome_path = '/clusterfs/nilah/oberon/dimelo_dev_source/test_inputs/genomes/chm13.draft_v1.0.fasta'
v1_1_genome_path = '/clusterfs/nilah/oberon/genomes/chm13.draft_v1.1.fasta'
hg38_genome_path = '/clusterfs/nilah/ayesha/basenji/baselines/gm12878/data/hg38.fa'

In [4]:
genome_fasta = pysam.FastaFile(v1_1_genome_path)

In [6]:
for chr_num in range(1,23):
    print(genome_fasta.get_reference_length(f'chr{chr_num}'))

248387328
242696752
201105948
193574945
182045439
172126628
160567428
146259331
150617247
134758134
135127769
133324548
113566686
101161492
99753195
96330374
84276897
80542538
61707364
66210255
45090682
51324926


In [63]:
cores = 32
bases_per_core = 100000000
chrom = 'chr1'
start_co = 40000000
end_co = 80000000
thresh = 128
sample_name = 'test'

In [64]:
start_time = time.time()
dm.parse_bam(
    megalodon_bam_filepath,
    sample_name,output_dir,
    basemods=['mA'],
    context_check_source = 'both',
    region=f'{chrom}:{start_co}-{end_co}',
    checkAgainstRef=True,
    pipeline='nanopore_megalodon',
    thresholds=[thresh,thresh,thresh],
    referenceGenome=genome_path,
    cores=cores,
    memory=cores*bases_per_core,
    formats_list=['bigwig']
)
end_time = time.time()
print('end-to-end time',end_time-start_time)

bases in batch (core allotment reached) 31196704
bases in batch (core allotment reached) 31202772
bases in batch (core allotment reached) 31206444
bases in batch (core allotment reached) 31195668
bases in batch (core allotment reached) 31207925
bases in batch (core allotment reached) 31242099
bases in batch (core allotment reached) 31238983
bases in batch (core allotment reached) 31201030
bases in batch (core allotment reached) 31213892
bases in batch (core allotment reached) 31211772
bases in batch (core allotment reached) 31195844
bases in batch (core allotment reached) 31199767
bases in batch (core allotment reached) 31256906
bases in batch (core allotment reached) 31202306
bases in batch (core allotment reached) 31237038
bases in batch (core allotment reached) 31285524
bases in batch (core allotment reached) 31209300
bases in batch (core allotment reached) 31202185
bases in batch (core allotment reached) 31199260
bases in batch (core allotment reached) 31217485
bases in batch (core

In [65]:
print('estimated total time for chr1 (min):',(end_time-start_time)*250000000/(end_co-start_co)/60)

estimated total time for chr1 (min): 27.05366591612498


In [61]:
with pyBigWig.open(output_dir+sample_name+'_bigwig_validmA.bw') as bw:
    contigs = bw.chroms()
    contig = list(contigs.keys())[0]
    contig_length = contigs[contig]
    values = bw.values(contig,start_co,end_co)
    validmA = np.array(values)
with pyBigWig.open(output_dir+sample_name+'_bigwig_read_depth.bw') as bw:
    contigs = bw.chroms()
    contig = list(contigs.keys())[0]
    contig_length = contigs[contig]
    values = bw.values(contig,start_co,end_co)
    read_depth = np.array(values)

In [62]:
print('average read depth 1Mb:',np.nanmean(read_depth))
print('adenine fraction over expected 25%',np.nanmean(validmA)/np.nanmean(read_depth)/0.295)

average read depth 1Mb: 3.7208340345855073
adenine fraction over expected 25% 0.9486016344732235
